In [1]:
import re
import PyPDF2
import pandas as pd

# Define a regular expression to match tables
table_regex = r'(?s)\b(?:\w+\s+){2,}\w+\b(?:\s*[,;]\s*\b(?:\w+\s+){2,}\w+\b)*'

# Open the PDF file in read-binary mode
with open('example.pdf', 'rb') as f:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(f)
    
    # Get the number of pages in the PDF file
    num_pages = len(pdf_reader.pages)
    
    # Loop through each page in the PDF file
    for page_num in range(num_pages):
        # Get the current page object
        page = pdf_reader.pages[page_num]
        
        # Extract the text from the current page
        page_text = page.extract_text()
        
        # Find all tables in page_text
        tables = re.findall(table_regex, page_text)
        
        # Loop through each table and create a pandas DataFrame
        for table in tables:
            # Split the table into rows
            rows = table.strip().split('\n')
            
            # Split the rows into cells
            cells = [row.split('|') for row in rows]
            
            # Remove leading and trailing whitespace from cells
            cells = [[cell.strip() for cell in row] for row in cells]
            
            # Remove empty rows and columns
            cells = [[cell for cell in row if cell] for row in cells if row]
            
            # Create a pandas DataFrame from the cells
            df = pd.DataFrame(cells[1:], columns=cells[0])
            
            # TODO: Clean and manipulate the df as needed

In [3]:
tables

['Observacion Linea\n Descripcion Producto\n Cantidad\n Emisor',
 'Rprieto\n Domicilio Comercial',
 'LOTE 44 PARQUE INDUSTRIAL 0  44\n RECONQUISTA',
 'SANTA FE\n Razón Social',
 'ALBERTO PRIETO S',
 'Fecha de Entrega',
 '2025\n Fecha de Emisión',
 '2025\n 152\n Orden de Producción\n N',
 'Toneladas\n Codigo\n 6\n Desarrollo',
 'FC633\n 6000\n ES EL ITEM QUE QUEDO PENDIENTE DEL\n 16',
 '01\n KG\n 2\n ENGORDE BOVINO 25 Kg\n EB25\n 80\n \n UN\n 28\n ENGORDE BOVINOS Granel\n EB0\n 28000\n SILO 1\n KG\n 5\n DESTETE PRECOZ MELAZA 25 Kg\n DPM25\n 200\n SACAR UN BOLSON\n UN\n 30\n CRECIMIENTO MELAZA Granel\n CM0\n 30000\n SILO 1\n KG\n 3\n RECRIA 14',
 'SIN PELLETEAR 25\n Kg\n R14SP25\n 120\n \n UN\n \n EXPELLER DE SOJA Molido 25 Kg\n EXPM25\n 40\n \n UN\n 74\n Total de\n Toneladas']

In [12]:
import pandas as pd
import pdfplumber

# Ruta del archivo PDF
pdf_path = "example.pdf"

# Lista para almacenar datos extraídos
data = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        print(text)
        if text:
            lines = text.split("\n")
            # print(lines)
            # Buscar información clave
            for line in lines:
                if "Fecha de Entrega:" in line:
                    fecha_entrega = line.split(":")[1].strip()
                elif "Fecha de Emisión:" in line:
                    fecha_emision = line.split(":")[1].strip()
                elif "Total de Toneladas:" in line:
                    total_toneladas = line.split(":")[1].strip()
                else:
                    # Extraer productos y cantidades
                    parts = line.split()
                    print(parts)
                    if len(parts) > 2 and parts[0].isdigit():
                        producto = " ".join(parts[1:-2])
                        codigo = parts[-2]
                        cantidad = parts[-1]
                        data.append([producto, codigo, cantidad])
                        print(data)

# Crear DataFrame
df = pd.DataFrame(data, columns=["Producto", "Código", "Cantidad"])

# # Agregar fechas y total de toneladas al DataFrame
# df["Fecha de Emisión"] = fecha_emision
# df["Fecha de Entrega"] = fecha_entrega
# df["Total Toneladas"] = total_toneladas



Orden de Producción
N° O.P: 152
Fecha de Emisión: 16/1/2025
Fecha de Entrega: 17/1/2025
Razón Social: ALBERTO PRIETO S.A.
Domicilio Comercial: LOTE 44 PARQUE INDUSTRIAL 0 44 Emisor: Rprieto
RECONQUISTA - SANTA FE
Observacion:
Codigo Cantidad Descripcion Producto Toneladas Observacion Linea
ES EL ITEM QUE QUEDO PENDIENTE DEL
FC633 6000 KG Desarrollo (La Malvina) 6
16/01
EB25 80 UN ENGORDE BOVINO 25 Kg 2
EB0 28000 KG ENGORDE BOVINOS Granel 28 SILO 1
DPM25 200 UN DESTETE PRECOZ MELAZA 25 Kg 5 SACAR UN BOLSON
CM0 30000 KG CRECIMIENTO MELAZA Granel 30 SILO 1
RECRIA 14% SIN PELLETEAR 25
R14SP25 120 UN 3
Kg
EXPM25 40 UN EXPELLER DE SOJA Molido 25 Kg
Total de 74
Toneladas:
['Orden', 'de', 'Producción']
['N°', 'O.P:', '152']
['Razón', 'Social:', 'ALBERTO', 'PRIETO', 'S.A.']
['Domicilio', 'Comercial:', 'LOTE', '44', 'PARQUE', 'INDUSTRIAL', '0', '44', 'Emisor:', 'Rprieto']
['RECONQUISTA', '-', 'SANTA', 'FE']
['Observacion:']
['Codigo', 'Cantidad', 'Descripcion', 'Producto', 'Toneladas', 'Observac

In [15]:
import pdfplumber
import pandas as pd

# Ruta del archivo PDF
pdf_path = "example.pdf"  # Asegúrate de que la ruta es correcta

# Lista para almacenar los datos extraídos
data = []

# Abrir el PDF y extraer texto
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            lines = text.split("\n")  # Dividir por líneas
            for line in lines:
                parts = line.split()  # Dividir por espacios
                print(parts)
                # Detectar líneas con datos de productos (ejemplo basado en el PDF proporcionado)
#                 if len(parts) >= 3 and parts[0].isdigit():  # Línea con número inicial
#                     toneladas = parts[0]
#                     descripcion = " ".join(parts[1:-2])  # Unir palabras del nombre
#                     codigo = parts[-2]
#                     cantidad = parts[-1]

#                     data.append([toneladas, descripcion, codigo, cantidad])

# # Convertir a DataFrame
# df = pd.DataFrame(data, columns=["Toneladas", "Descripción", "Código", "Cantidad"])

# # Convertir tipos de datos
# df["Toneladas"] = pd.to_numeric(df["Toneladas"], errors="coerce")
# df["Cantidad"] = pd.to_numeric(df["Cantidad"], errors="coerce")

# # Mostrar el DataFrame
# print(df)

# # Guardar a CSV o Excel si es necesario
# df.to_csv("productos.csv", index=False)
# df.to_excel("productos.xlsx", index=False)


['Orden', 'de', 'Producción']
['N°', 'O.P:', '152']
['Fecha', 'de', 'Emisión:', '16/1/2025']
['Fecha', 'de', 'Entrega:', '17/1/2025']
['Razón', 'Social:', 'ALBERTO', 'PRIETO', 'S.A.']
['Domicilio', 'Comercial:', 'LOTE', '44', 'PARQUE', 'INDUSTRIAL', '0', '44', 'Emisor:', 'Rprieto']
['RECONQUISTA', '-', 'SANTA', 'FE']
['Observacion:']
['Codigo', 'Cantidad', 'Descripcion', 'Producto', 'Toneladas', 'Observacion', 'Linea']
['ES', 'EL', 'ITEM', 'QUE', 'QUEDO', 'PENDIENTE', 'DEL']
['FC633', '6000', 'KG', 'Desarrollo', '(La', 'Malvina)', '6']
['16/01']
['EB25', '80', 'UN', 'ENGORDE', 'BOVINO', '25', 'Kg', '2']
['EB0', '28000', 'KG', 'ENGORDE', 'BOVINOS', 'Granel', '28', 'SILO', '1']
['DPM25', '200', 'UN', 'DESTETE', 'PRECOZ', 'MELAZA', '25', 'Kg', '5', 'SACAR', 'UN', 'BOLSON']
['CM0', '30000', 'KG', 'CRECIMIENTO', 'MELAZA', 'Granel', '30', 'SILO', '1']
['RECRIA', '14%', 'SIN', 'PELLETEAR', '25']
['R14SP25', '120', 'UN', '3']
['Kg']
['EXPM25', '40', 'UN', 'EXPELLER', 'DE', 'SOJA', 'Molido', '2

In [2]:
import pyodbc
import pandas as pd

# Datos de conexión
server = "SERVIDORII/MANTIS"
database = "mantis"
usuario = "sa"  # Opcional si usas autenticación SQL Server
password = "mantis44*"

# Cadena de conexión (para autenticación SQL Server)
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={usuario};PWD={password}'

# Si usas autenticación de Windows (SSPI)
# conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes'

# Conectar a la base de datos
conn = pyodbc.connect(conn_str)
cur = conn.cursor()

# # Ejecutar una consulta
# query = "SELECT TOP 10 * FROM tabla_ejemplo"
# df = pd.read_sql(query, conn)

# # Mostrar el DataFrame
# print(df)

# # Cerrar la conexión
# cur.close()
# conn.close()


OperationalError: ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]No existe el servidor SQL Server o se ha denegado el acceso al mismo. (17) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionOpen (Connect()). (67)')